In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import pickle
import json 
pd.set_option('display.max_columns', None)

In [2]:
f = open('Full_stops_sequence_merged.json',)
Stops_Seq = json.load(f)

In [13]:
def get_route(stops_sequence, bus_route):
    route_dict = {}
    for i in range(0, 2):

        if (bus_route + "_{}".format(i)) in stops_sequence:

            sequence = stops_sequence[bus_route + "_{}".format(i)]
            route_dict['d{}'.format(i)] = {}
            route_dict['d{}'.format(i)] = sequence

        else:
            pass

    return route_dict

In [4]:
def peak_hour(hour):
    if (7 < hour < 10) or (16 < hour < 19):
        peak = 1
    else:
        peak = 0

    return peak

In [5]:
def direction(start, end, stops):

    """ deciphers which direction the journey is taking place and returns the relevant key
    such that the dictionary can be accessed """

    for key, value in stops.items():
        if start in value and end in value:
            return key

In [80]:
def prediction(route, hour, day, month, start_stop_id, end_stop_id, wind_speed, temp, humidity, weather_main,
               stops):

    journey_stops = stops[direction(start_stop_id, end_stop_id, stops)]
    stop_sequence = journey_stops[journey_stops.index(start_stop_id):journey_stops.index(end_stop_id) + 1]

    print(stop_sequence)
    prediction_df = pd.DataFrame(stop_sequence, columns=['STOPPOINTID'])
    prediction_df['NEXT_STOPPOINTID'] = prediction_df['STOPPOINTID'].shift(-1)
    prediction_df = prediction_df.dropna(axis=0)
    prediction_df['NEXT_STOPPOINTID'] = prediction_df['NEXT_STOPPOINTID'].astype('int')

    df = pd.DataFrame(columns=['DAY', 'HOUR', 'MONTH', 'PEAK_HOUR', 'wind_speed', 'temp', 'humidity', 'weather_main'])

    user_input = [day, hour, month, peak_hour(hour), wind_speed, temp, humidity, weather_main]

    df.loc[0] = user_input

    frames = [prediction_df, df]
    result = pd.concat(frames, axis=1).ffill()

    result['STOPPOINTID'] = result['STOPPOINTID'].astype('category')
    result['NEXT_STOPPOINTID'] = result['NEXT_STOPPOINTID'].astype('category')
    result['DAY'] = result['DAY'].astype('category')
    result['HOUR'] = result['HOUR'].astype('category')
    result['MONTH'] = result['MONTH'].astype('category')
    result['PEAK_HOUR'] = result['PEAK_HOUR'].astype('int')
    result['wind_speed'] = result['wind_speed'].astype('float')
    result['temp'] = result['temp'].astype('int')
    result['humidity'] = result['humidity'].astype('float')
    result['weather_main'] = result['weather_main'].astype('category')

    df_dummies = pd.get_dummies(result)

    with open('/home/tomah/data/Model_Testing/LinRegPickles/LinReg_route_{}_headers.pkl'.format(route), 'rb') as handle:
        dummies = pickle.load(handle)
    full_dummies = pd.DataFrame(columns=dummies)

    df_a, df_b = full_dummies.align(df_dummies, fill_value=0)

    final_df_for_predict = df_b.reindex(full_dummies.columns, axis=1)

    with open('/home/tomah/data/Model_Testing/LinRegPickles/LinReg_Model_route_{}.pkl'.format(route), 'rb') as handle:
        model = pickle.load(handle)

    y_pred_linear = model.predict(final_df_for_predict)

    # remove any predictions that are way off due to the new stop sequence data being used
    # take the average journey time between all other stops and replace the incorrect prediction with this value
    
    good_array = [item for item in y_pred_linear if item < 400 and item >0]
    average = sum(good_array)/len(good_array)
    good_pred = [average if test > 400 or test < 0 else test for test in y_pred_linear]
    
    print(y_pred_linear, '\n')
    print(good_pred)
    
    final_prediction = sum(good_pred) / 60

    return final_prediction

In [31]:
stops_dict = get_route(Stops_Seq, '16')

In [32]:
stops_dict

{'d0': [7347,
  3669,
  7349,
  1631,
  1632,
  5053,
  1633,
  1634,
  1635,
  1636,
  1637,
  1638,
  1639,
  1640,
  231,
  232,
  233,
  234,
  235,
  236,
  237,
  213,
  214,
  4432,
  119,
  44,
  7603,
  45,
  46,
  47,
  48,
  49,
  51,
  52,
  262,
  270,
  336,
  1279,
  1282,
  4456,
  1284,
  1285,
  1287,
  1288,
  1289,
  1290,
  1291,
  1292,
  1293,
  1294,
  1295,
  1296,
  1297,
  1298,
  1299,
  1300,
  1301,
  1302,
  1303,
  1304,
  1305,
  1320,
  1321,
  1322,
  1323,
  1324,
  7068,
  2966,
  2967,
  2968,
  2969,
  2970,
  2971,
  5051,
  5052,
  2975],
 'd1': [5171,
  2976,
  2977,
  2978,
  2979,
  2980,
  2981,
  2991,
  2992,
  1325,
  1326,
  1327,
  1328,
  1329,
  1330,
  1331,
  1332,
  1333,
  1334,
  7293,
  1335,
  1336,
  1337,
  1338,
  1339,
  1340,
  1341,
  1342,
  1343,
  1344,
  1345,
  1347,
  1348,
  1349,
  1350,
  1352,
  1353,
  1355,
  1357,
  1359,
  320,
  278,
  8,
  12,
  14,
  15,
  17,
  18,
  19,
  21,
  7602,
  85,
  203,
  204,

In [27]:
d = direction(85, 215, stops_dict)

In [28]:
print(d)

d1


In [81]:
prediction('16', 10, 1, 8, 1342, 7347, 8, 11, 11, 'Rain', stops_dict)

[1342, 1343, 1344, 1345, 1347, 1348, 1349, 1350, 1352, 1353, 1355, 1357, 1359, 320, 278, 8, 12, 14, 15, 17, 18, 19, 21, 7602, 85, 203, 204, 205, 215, 216, 217, 218, 219, 220, 1622, 1623, 1624, 1625, 1626, 1627, 1628, 1629, 1630, 7347]
[2.10050049e+01 7.32644653e+01 4.20750122e+01 7.12653198e+01
 8.18760376e+01 3.21667480e+01 4.33318481e+01 8.99859009e+01
 3.35797119e+01 4.84957170e+07 7.44408569e+01 1.19468811e+02
 9.44899292e+01 1.85909790e+02 7.96805204e+07 1.14799927e+02
 5.14135132e+01 5.51184692e+01 7.21540527e+01 5.39656372e+01
 2.78997803e+01 6.03074951e+01 3.06143188e+01 4.27145386e+01
 6.78944702e+01 5.44329224e+01 3.17337036e+01 9.41070557e+01
 7.20065308e+01 2.32195435e+01 8.32307739e+01 5.50765381e+01
 9.49637451e+01 4.44577637e+01 2.44042969e+01 7.60714722e+01
 8.86654663e+01 5.10200806e+01 5.77492676e+01 4.37208252e+01
 3.49431152e+01 6.98836060e+01 1.93529114e+02] 

[21.0050048828125, 73.26446533203125, 42.07501220703125, 71.26531982421875, 81.87603759765625, 32.16674804

47.77120760785854

# exploring solution for incorrect predictions 



In [47]:
test_array

array([2.10050049e+01, 7.32644653e+01, 4.20750122e+01, 7.12653198e+01,
       8.18760376e+01, 3.21667480e+01, 4.33318481e+01, 8.99859009e+01,
       3.35797119e+01, 4.84957170e+07, 7.44408569e+01, 1.19468811e+02,
       9.44899292e+01, 1.85909790e+02, 7.96805204e+07, 1.14799927e+02,
       5.14135132e+01, 5.51184692e+01, 7.21540527e+01, 5.39656372e+01,
       2.78997803e+01, 6.03074951e+01, 3.06143188e+01, 4.27145386e+01,
       6.78944702e+01, 5.44329224e+01, 3.17337036e+01, 9.41070557e+01,
       7.20065308e+01, 2.32195435e+01, 8.32307739e+01, 5.50765381e+01,
       9.49637451e+01, 4.44577637e+01, 2.44042969e+01, 7.60714722e+01,
       8.86654663e+01, 5.10200806e+01, 5.77492676e+01, 4.37208252e+01,
       3.49431152e+01, 6.98836060e+01, 1.93529114e+02])

We're going to replace any predictions that are over 400 seconds with the average jounrye time between stops.

In [67]:
good_array = [item for item in test_array if item < 400 and item >0]

In [68]:
average = sum(good_array)/len(good_array)

In [69]:
average

66.6574989877096

In [74]:
new_pred = [average if test > 400 or test < 0 else test for test in test_array]

In [75]:
new_pred

[21.0050048828125,
 73.26446533203125,
 42.07501220703125,
 71.26531982421875,
 81.87603759765625,
 32.166748046875,
 43.33184814453125,
 89.98590087890625,
 33.5797119140625,
 66.6574989877096,
 74.44085693359375,
 119.46881103515625,
 94.48992919921875,
 185.9097900390625,
 66.6574989877096,
 114.7999267578125,
 51.41351318359375,
 55.11846923828125,
 72.154052734375,
 53.96563720703125,
 27.8997802734375,
 60.3074951171875,
 30.61431884765625,
 42.71453857421875,
 67.89447021484375,
 54.43292236328125,
 31.73370361328125,
 94.1070556640625,
 72.00653076171875,
 23.21954345703125,
 83.23077392578125,
 55.0765380859375,
 94.9637451171875,
 44.457763671875,
 24.404296875,
 76.07147216796875,
 88.66546630859375,
 51.02008056640625,
 57.749267578125,
 43.7208251953125,
 34.943115234375,
 69.88360595703125,
 193.52911376953125]